UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 122286: invalid continuation byte

In [ ]:
import pandas as pd
import os
import re
import snowflake.connector
from snowflake.snowpark import Session


from snowflake.connector.pandas_tools import write_pandas




session = Session.builder.configs({
    "connection_name": "myconn"
}).create()

# Test query
df = session.sql("SELECT CURRENT_USER(), CURRENT_DATABASE(), CURRENT_SCHEMA()").to_pandas()
print(df)


conn = snowflake.connector.connect(
    connection_name="myconn"
)

cur = conn.cursor()

# cur.execute("""PUT file://C:/data/moontea.csv
# @PROJECT_DB.DATASETS.MY_STAGE
# AUTO_COMPRESS=TRUE
# OVERWRITE=TRUE""")

# df = cur.fetch_pandas_all()
# print(df)




# write_pandas(
#     conn,
#     df,
#     table_name="MY_TABLE",
#     database="PROJECT_DB",
#     schema="DATASETS"
# )


# === CONFIG ===



files = [r"C:\Users\apagano\Downloads\COUNTY_ZIP_122025.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_032020.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_062020.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_092020.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_122020.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_032021.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_062021.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_092021.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_122021.xlsx",
r"C:\Users\apagano\Downloads\county_zip_032022.xlsx",
r"C:\Users\apagano\Downloads\county_zip_062022.xlsx",
r"C:\Users\apagano\Downloads\county_zip_092022.xlsx",
r"C:\Users\apagano\Downloads\county_zip_122022.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_032023.xlsx",
r"C:\Users\apagano\Downloads\county_zip_062023.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_092023.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_122023.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_032024.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_062024.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_092024.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_122024.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_032025.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_062025.xlsx",
r"C:\Users\apagano\Downloads\COUNTY_ZIP_092025.xlsx",]


test_file = [r"C:\Users\apagano\Downloads\COUNTY_ZIP_122025.xlsx",]

STAGE = '@JBS_DATASETS.AREA_HEALTH_RESOURCE_FILE_AHRF.AREA_HEALTH_RESOURCE_FILE_AHRF_STAGE'
FILE_FORMAT = 'CSV_FORMAT'
DATABASE = 'JBS_DATASETS'
SCHEMA = 'AREA_HEALTH_RESOURCE_FILE_AHRF'

# === Helpers ===
def clean_name(s):
    s = re.sub(r"[^\w]+", "_", s)       # replace non-alphanum with underscore
    return re.sub(r'_+', '_', s).strip('_').upper()

def clean_filename(file_path, sheet=None):
    base = os.path.splitext(os.path.basename(file_path))[0]
    if sheet: base += f"_{sheet}"
    return clean_name(base) + '.csv'

def process_df(df, path):
    df.columns = [clean_name(c) for c in df.columns]
    df.to_csv(path, index=False)
    return df

# === Process files ===
cleaned = []

for f in files:
    ext = os.path.splitext(f)[1].lower()
    
    # --- Process CSV / TSV files ---
    if ext in ['.csv', '.tsv']:
        sep = ',' if ext == '.csv' else '\t'
        try:
            df = pd.read_csv(f, sep=sep, low_memory=False)
        except UnicodeDecodeError:
            # Fallback to latin1 encoding if UTF-8 fails
            df = pd.read_csv(f, sep=sep, encoding='latin1', low_memory=False)
        except Exception as e:
            print(f"Error reading {f}: {e}")
            continue
        
        out = os.path.splitext(f)[0] + "_cleaned.csv"
        process_df(df, out)
        cleaned.append((out, clean_filename(out)))
        print(f"Processed CSV/TSV: {f}")
    
    # --- Process XLSX files ---
    elif ext == '.xlsx':
        try:
            xls = pd.ExcelFile(f)
        except Exception as e:
            print(f"Error opening Excel file {f}: {e}")
            continue
        
        for sh in xls.sheet_names:
            try:
                df = pd.read_excel(xls, sheet_name=sh)
            except Exception as e:
                print(f"Error reading sheet {sh} in {f}: {e}")
                continue
            
            clean_sh = clean_name(sh)
            out = os.path.splitext(f)[0] + f"_{clean_sh}_cleaned.csv"
            process_df(df, out)
            cleaned.append((out, clean_filename(f, clean_sh)))
            print(f"Processed sheet '{sh}' in Excel: {f}")


# === Snowflake PUT statements ===
print("\n-- PUT statements --")
for local, stage_file in cleaned:
    path = os.path.abspath(local).replace("\\","/")
    print(f'snow sql -q "PUT \'file://{path}\' {STAGE}/{stage_file} AUTO_COMPRESS=FALSE OVERWRITE=TRUE;"')
    cur.execute(f"""PUT \'file://{path}\' {STAGE}/{stage_file} AUTO_COMPRESS=FALSE OVERWRITE=TRUE;""")
# === CREATE TABLE + COPY INTO ===
print("\n-- CREATE TABLE & COPY INTO --")

for local, tbl_full in cleaned:
    df = pd.read_csv(local, nrows=0)
    cols = ",\n    ".join([f'"{c}" VARCHAR' for c in df.columns])
    tbl = tbl_full.removesuffix(".csv")
    create = f"CREATE OR REPLACE TABLE {DATABASE}.{SCHEMA}.{tbl} (\n    {cols}\n);"
    copy = f"""
COPY INTO {DATABASE}.{SCHEMA}.{tbl}
FROM {STAGE}/{tbl_full}
FILE_FORMAT = (FORMAT_NAME = {FILE_FORMAT})
ON_ERROR = 'CONTINUE';
""".strip()
    print(f"\n-- {tbl} --\n{create}\n{copy}\n")
    session.sql(f"\n-- {tbl} --\n{create}\n{copy}\n").collect()


cur.close()
conn.close()


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/50265338-cd9a-41ef-9f86-110fc45ce026/saml2?SAMLRequest=lVJdb9sgFP0rFnu2ASfxEpSkyoe6Zmo6q0mnbW8EY5cGgwO4bvrrh51Eah9aqW%2FAPeeewz13fPVSyuCZGyu0mgAcIRBwxXQmVDEBD9vrcAgC66jKqNSKT8CRW3A1HVtayorMaveo7vmh5tYFvpGypC1MQG0U0dQKSxQtuSWOkc1sfUviCBFqLTfOy4EzJbPCaz06VxEIm6aJml6kTQFjhBBEI%2BhRLeQbeCNRfa5RGe000%2FJCefF%2F%2BkACQ9RvJTzCK6Rn4lyo0wg%2BU9mdQJbcbLdpmP7abEEwu%2FxuoZWtS2423DwLxh%2Fub08GrHdw0PxYvNqQ7ncYj4bfI6t0k0u650yXVe1808ifYM4zKHUh%2FKhWywmo9iJL7g5q16TN3yJt%2FMVt1k838%2BWfdYL1j91gXaTyaTf%2F6RZ4wxgIfl%2BCjdtgV9bWfKXaOJ1%2FQnESojjEaBtj0otJ3I%2F6SfIPBEsfp1DUdcyL585HVApmtNW500oKxTuXA99o0OsNQ5aNaNjHPA9H%2BTAJMUY56w8Y93XYhhaD0%2BKQzoiZfnEcY%2FiWfF7BO5%2FKaplqKdgxuNampO7j0HCEuxeRhXkHJbykQs6yzHBrfXhS6mZhOHV%2B052pOYDTk%

ProgrammingError: 002003 (02000): SQL compilation error:
Stage 'PROJECT_DB.DATASETS.MY_STAGE' does not exist or not authorized.

In [4]:
import shutil
from pathlib import Path

def move_csv_xlsx_files(input_path):
    input_path = Path(input_path)
    
    if not input_path.exists():
        print(f"The path {input_path} does not exist.")
        return
    
    # Create the destination folder next to input_path
    dest_folder = input_path.parent / "csv_xlsx"
    dest_folder.mkdir(exist_ok=True)
    
    # Walk recursively through the input path
    for file_path in input_path.rglob("*"):
        if file_path.is_file() and file_path.suffix.lower() in [".csv", ".xlsx"]:
            try:
                shutil.move(str(file_path), dest_folder / file_path.name)
                print(f"Moved: {file_path} -> {dest_folder / file_path.name}")
            except Exception as e:
                print(f"Error moving {file_path}: {e}")

# Example usage
input_folder = r"C:\Users\apagano\Downloads\AHRF\Extracted"
move_csv_xlsx_files(input_folder)


Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 CSV\AHRF 2023-2024 CSV\ahrf2024_Feb2025.csv -> C:\Users\apagano\Downloads\AHRF\csv_xlsx\ahrf2024_Feb2025.csv
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 CSV\AHRF 2023-2024 CSV\CSV Files by Categories\ahrf2024env.csv -> C:\Users\apagano\Downloads\AHRF\csv_xlsx\ahrf2024env.csv
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 CSV\AHRF 2023-2024 CSV\CSV Files by Categories\ahrf2024exp.csv -> C:\Users\apagano\Downloads\AHRF\csv_xlsx\ahrf2024exp.csv
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 CSV\AHRF 2023-2024 CSV\CSV Files by Categories\ahrf2024geo_Feb2025.csv -> C:\Users\apagano\Downloads\AHRF\csv_xlsx\ahrf2024geo_Feb2025.csv
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 CSV\AHRF 2023-2024 CSV\CSV Files by Categories\ahrf2024hf.csv -> C:\Users\apagano\Downloads\AHRF\csv_xlsx\ahrf2024hf.csv
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 C

In [5]:
import shutil
from pathlib import Path

def move_other_files(input_path):
    input_path = Path(input_path)
    
    if not input_path.exists():
        print(f"The path {input_path} does not exist.")
        return
    
    # Create the destination folder next to input_path
    dest_folder = input_path.parent / "other_files"
    dest_folder.mkdir(exist_ok=True)
    
    # Walk recursively through the input path
    for file_path in input_path.rglob("*"):
        if file_path.is_file() and file_path.suffix.lower() not in [".csv", ".xlsx"]:
            try:
                shutil.move(str(file_path), dest_folder / file_path.name)
                print(f"Moved: {file_path} -> {dest_folder / file_path.name}")
            except Exception as e:
                print(f"Error moving {file_path}: {e}")

# Example usage
input_folder = r"C:\Users\apagano\Downloads\AHRF\Extracted"
move_other_files(input_folder)


Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 CSV\AHRF 2023-2024 CSV\2023-2024_AHRFDUA.docx -> C:\Users\apagano\Downloads\AHRF\other_files\2023-2024_AHRFDUA.docx
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 User Tech\AHRF 2023-2024 User Tech\AHRF USER 2023-2024_Feb2025.docx -> C:\Users\apagano\Downloads\AHRF\other_files\AHRF USER 2023-2024_Feb2025.docx
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF 2023-2024 User Tech\AHRF 2023-2024 User Tech\What's New 2023-2024 AHRF_Feb2025.pdf -> C:\Users\apagano\Downloads\AHRF\other_files\What's New 2023-2024 AHRF_Feb2025.pdf
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF SN 2023-2024 CSV\AHRF SN 2023-2024 CSV\2023-2024_AHRFDUA.docx -> C:\Users\apagano\Downloads\AHRF\other_files\2023-2024_AHRFDUA.docx
Moved: C:\Users\apagano\Downloads\AHRF\Extracted\AHRF SN 2023-2024 User Tech\AHRF SN 2023-2024 User Tech\AHRF SN User 2023-2024.docx -> C:\Users\apagano\Downloads\AHRF\other_files\AHRF SN User 2023-20